In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

/home/robin/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/robin/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/robin/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
""" Données """
data = pd.read_csv('dryad_zebra.csv')
position = np.array(data[['x', 'y']])
Y = np.array(data[['speed', 'angle']])
Y = (Y - np.mean(Y, axis=0)) / np.sqrt(np.var(Y, axis=0))

In [3]:
""" Parms """
nu = np.array([1, 1, 1])/3.0
pi = np.array([[0.8, 0.1, 0.1], [0.1, 0.8, 0.1], [0.1, 0.1, 0.8]])
mu = np.array([[-0.5, -0.5], [-1, 1], [1, -0.5]])
sigma = np.array([[[0.5, 0], [0, 0.5]], [[0.5, 0], [0, 0.5]], [[0.5, 0], [0, 0.5]]])
parms = (nu, pi, mu, sigma)
parms

(array([0.33333333, 0.33333333, 0.33333333]),
 array([[0.8, 0.1, 0.1],
        [0.1, 0.8, 0.1],
        [0.1, 0.1, 0.8]]),
 array([[-0.5, -0.5],
        [-1. ,  1. ],
        [ 1. , -0.5]]),
 array([[[0.5, 0. ],
         [0. , 0.5]],
 
        [[0.5, 0. ],
         [0. , 0.5]],
 
        [[0.5, 0. ],
         [0. , 0.5]]]))

In [4]:
""" Emissions """
K, p = np.shape(mu)
phi = np.transpose([multivariate_normal(mean=mu[g], cov=sigma[g]).pdf(Y) for g in range(K)])


In [5]:
""" Forward """
n, p = np.shape(phi)
forward = np.zeros([n, K])
forward[0] = nu * phi[0]
logL = np.log(sum(forward[0]))
forward[0] = forward[0] / np.sum(forward[0])
for i in np.arange(1, n):
    forward[i] = np.matmul(forward[i-1], pi)
    forward[i] = forward[i] * phi[i]
    logL = logL + np.log(np.sum(forward[i]))
    forward[i] = forward[i] / np.sum(forward[i])

In [6]:
""" Backward """
tau  = np.zeros([n, K])
G = np.zeros([n, K])
eta = np.zeros([K, K])
tau[n-1] = forward[n-1]
for i in np.arange(n-2, -1, -1):
    G[i+1] = np.matmul(forward[i], pi)
    B = tau[i+1] / G[i+1]
    tau[i] = forward[i,] * np.matmul(pi, B)
    tau[i] <- tau[i] / np.sum(tau[i])
    eta  = eta + pi * np.matmul(forward[i], B)
eta

array([[148.36769785,  18.54596223,  18.54596223],
       [ 18.54596223, 148.36769785,  18.54596223],
       [ 18.54596223,  18.54596223, 148.36769785]])